Thankfully [tafsir_api](https://github.com/spa5k/tafsir_api) has many tafsirs sectioned per Ayah. The 114 for Waseet have been downloaded and will be merged here:

In [1]:
import glob
import json

# Get list of JSON file paths sorted by filename
jsons = sorted(glob.glob('Tafsirs/*.json'), key=lambda x: int(x.split("/")[-1].split(".")[0]))

full_tafsir = []
for fname in jsons:
    with open(fname) as infile:
        tafsir = json.load(infile)
        tafsir = sorted(tafsir["ayahs"], key=lambda x: x["ayah"])
        full_tafsir.append(tafsir)  

# Save the merged JSON to a file
with open('tafsirs.json', 'w') as f:
    json.dump(full_tafsir, f, ensure_ascii=False, indent=4)


Now let's read the tafsir json and make some transformations for readability.

Remove extra characters (e.g., dots) between braces so no new lines are inserted there later:

In [2]:
def remove_chars_between_braces(text):
    inside_braces = False
    result = ''
    for char in text:
        if char == '{':
            inside_braces = True
        elif char == '}':
            inside_braces = False
        elif inside_braces and (char in ['.', ".", "_"]) or (char.isspace() and char !=" "):
            continue
        result += char
    return result

# Example usage:
arabic_string = "مرحبا. هذا هو النص العربي {مع. النقاط. ولكن. لا نريدها. هنا    }."
cleaned_string = remove_chars_between_braces(arabic_string)
print(cleaned_string)

مرحبا. هذا هو النص العربي {مع النقاط ولكن لا نريدها هنا    }.


Make the 12 (as in the if conditions) words before the colon italic and bold if they are at the start of the sentence. What italic will do is change the color (has a custom style in the code) but not make actual text italic

In [3]:
import re

def make_before_colons_italic(input_string):
    lines = input_string.split('<br><br>')
    processed_lines = []

    for line in lines:
        parts = line.split(':')
        if len(parts) > 1:
            before_colon = parts[0]
            words_before_colon = len(before_colon.split())
            if words_before_colon <= 12:
                line = '<b><i>' + before_colon.strip() + ':</i></b>' + ':'.join(parts[1:])
        processed_lines.append(line)

    return '<br><br>'.join(processed_lines)

Extra `<br><br>` tags (white space) will appear at the end of the tafsir after processing so this fixes it

In [4]:
def remove_final_br_tags(string):
    if string.endswith('<br><br>'):
        return string[:-8]  # Removing the last 8 characters ("<br><br>")
    else:
        return string

Can remove tashkeel for stemming (not really used here)

In [5]:
def remove_tashkeel(text):
    return re.sub(u'[\u064e\u064f\u0650\u0651\u0652\u064c\u064b\u064d\u0640\ufc62]','',text)

Special words that should be colored by wrapping in an italic tag

In [6]:
special_list = [
"سبحانه",
"المراد",
"الله",
"تعالى",
"سبحانه",
"صلى",
"قال",
"قوله",
"عليه",
"الآية",
"ابن",
"القرآن",
"والمراد",
"الرسول",
"وقوله",
"فقال",
"الكريم",
"موسى",
"النبى",
"وسلم",
"القيامة",
"الإِمام",
"بالله",
"سيدنا",
"محمد",
"آله",
"وعلى",
"وصحبه",
"وسلم",
"وصلى",
]

In [15]:
import json
import re
from tqdm.notebook import tqdm

# read tafsirs.json
with open('tafsirs.json') as f:
    tafsirs = json.load(f)
    
with open ('../suras.json') as f:
    suras = json.load(f) 

# Iterate through each list with tqdm
for i, surah in tqdm(enumerate(tafsirs), total=len(tafsirs)):
    for j, tafsir in enumerate(surah):
        tafsirWords = tafsir['text'].split(" ")
        
        # Any word just before : should be italic (i.e., be colored as we mentioned above)
        for k, tafsirWord in enumerate(tafsirWords[:-1]):
            if tafsirWords[k+1] == ":" :
                tafsir['text'] = re.sub(r'\b{}\b'.format(re.escape(tafsirWords[k])), f'<i>{tafsirWords[k]}</i>', tafsir['text'])
        
        # special words should be italic (i.e., be colored as we mentioned above)
        for word in special_list:
            pattern = f'(?<![^\W\d_])({word})(?![^\W\d_])'  # Match the word only if not preceded or followed by a non-alphanumeric character
            tafsir['text'] = re.sub(pattern, r'<i>\1</i>', tafsir['text'])


        # if you ever find {...} surround it with <b></b> to make it bold
        tafsir['text'] = tafsir['text'].replace('{', '{ <b>')
        tafsir['text'] = tafsir['text'].replace('}', '</b> }')
        tafsir['text'] = tafsir['text'].replace('(', '{ <b>')
        tafsir['text'] = tafsir['text'].replace(')', '</b> }')
        tafsir['text'] =  tafsir['text'].replace('<b> ', '<b>')
        tafsir['text'] =  tafsir['text'].replace(' </b>', '</b>')

        # remove any special characters within {.*} for purpose mentioned above
        tafsir['text'] = remove_chars_between_braces(tafsir['text'])

        # Add new lines after '. ' or '...' unless it's followed by a quote
        pattern = r'\.(?!")(?! ")(?!\.)(?! \.)'
        tafsir['text'] = re.sub(pattern, '.<br><br>', tafsir['text'])
        pattern = r'\.\.\.(?!")(?!\.)'
        tafsir['text'] = re.sub(pattern, '...<br><br>', tafsir['text'])

        # Remove spaces preceding a dot in the 'text' field
        tafsir['text'] = tafsir['text'].replace(' .', '.')
        
        # Remove floating dots
        tafsir['text'] = tafsir['text'].replace('<br>.<br>', '')

        # Make words before colon colored
        tafsir['text'] = make_before_colons_italic(tafsir['text'])

        # Remove extra new lines at the end
        tafsir['text'] =  remove_final_br_tags(tafsir['text'])
        
# save as surahTafsir
with open('surahTafsirs.json', 'w') as f:
    json.dump(tafsirs, f, ensure_ascii=False, indent=4) 

  0%|          | 0/114 [00:00<?, ?it/s]

Test parsing and reformatting:

In [13]:
from IPython.display import display_html

display_html(
    f"""<div style="direction:rtl">
    {tafsirs[17][19]['text']}
    </div>
    """
    , raw=True)

" إنهم إن يظهروا عليكم يرجموكم " أي إن علموا بمكانكم " يرجموكم أو يعيدوكم في ملتهم " يعنون أصحاب دقيانوس يخافون منهم أن يطلعوا على مكانهم فلا يزالون يعذبونهم بأنواع العذاب إلى أن يعيدوهم في ملتهم التي هم عليها أو يموتوا وإن وافقتموهم على العود في الدين فلا فلاح لكم في الدنيا ولا في الآخرة ولهذا قال " ولن تفلحوا إذا أبدًا".

In [14]:
tafsirs[14][0]['text']

'قد اختلف المفسرون في الحروف المقطعة التي في أوائل السور فمنهم من <i>قال</i> هي مما استأثر <i>الله</i> بعلمه فردوا علمها إلى <i>الله</i> ولم يفسرها حكاه القرطبي في تفسـره عن أبي بكر وعمر وعثمان وعلي وابن مسعود رضي <i>الله</i> عنهم أجمعين وقاله عامر الشعبي وسفيان الثوري والربيع بن خيثم واختاره أبو حاتم بن حبان.<br><br> ومنهم من فسرها واختلف هؤلاء في معناها <i>فقال</i> عبدالرحمن بن زيد بن أسلم إنما هي أسماء السور.<br><br> <i>قال</i> العلامة أبو القاسم محمود بن عمر الزمخشري في تفسيره وعليه إطباق الأكثر ونقل عن سيبويه أنه نص <i>عليه</i> ويعتضد لهذا بما ورد في الصحيحين عن أبي هريرة أن رسول <i>الله</i> <i>صلى</i> <i>الله</i> <i>عليه</i> <i><i>وسلم</i></i> كان يقرأ في صلاة الصبح يوم الجمعة "الم" السجدة و "هل أتى على الإنسان" وقال سفيان الثوري عن <i>ابن</i> أبي نجيح عن مجاهد أنه <i>قال</i>: الم وحم والمص وص.<br><br> فواتح افتتح <i>الله</i> بها <i>القرآن</i> وكذا <i>قال</i> غيره عن مجاهد وقال مجاهد في رواية أبي حذيفة <i>موسى</i> بن مسعود عن شبل عن <i>ابن</i> أبي نجيح عنه أنه <i>قال</i> الم اسم 

Get most frequest words to then add in `special_words` list above.

In [10]:
from collections import Counter
from tqdm import tqdm

# Concatenate all text
all_text = ' '.join([tafsir['text'] for surah in tafsirs for tafsir in surah])

# Tokenize the text into words
words = all_text.split()

# Count the frequency of each word
word_counts = Counter(words)

# Sort the words based on their frequencies
sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Select the top 200 words
top_200_words = sorted_words[:200]

# Print the top 200 words
for i, (word, freq) in tqdm(enumerate(top_200_words), total=200):
    print(f"{i+1}. {word}")

100%|██████████| 200/200 [00:00<00:00, 328836.06it/s]

1. <i>الله</i>
2. من
3. عن
4. بن
5. في
6. <i>قال</i>
7. <i>عليه</i>
8. <i>ابن</i>
9. على
10. أن
11. حدثنا
12. "
13. أبي
14. <i>صلى</i>
15. ما
16. <i><i>وسلم</i></i>
17. وقال
18. لا
19. أي
20. <i>تعالى</i>
21. رسول
22. إلى
23. ثم
24. ذلك
25. ولا
26. هذا
27. <i>فقال</i>
28. به
29. كان
30. له
31. إلا
32. <i>قال</i>:
33. أبو
34. كما
35. <i>محمد</i>
36. إن
37. عباس
38. يا
39. الذي
40. وهو
41. هو
42. حتى
43. وقد
44. أنه
45. -
46. أو
47. رضي
48. عنه
49. هذه
50. يوم
51. إذا
52. لم
53. <i>الآية</i>
54. يقول
55. عبدالله
56. <i>وقوله</i>
57. قد
58. لهم
59. <i>قوله</i>
60. فيه
61. الذين
62. كل
63. حديث
64. رواه
65. النبي
66. أحمد
67. وما
68. سعيد
69. علي
70. الناس
71. والله
72. جرير
73. عبد
74. عليهم
75. أهل
76. الإمام
77. عمر
78. فيها
79. وهذا
80. يعني
81. لما
82. بعد
83. بين
84. عند
85. عز
86. ومن
87. <i>موسى</i>
88. الأرض
89. فإن
90. ولهذا
91. بها
92. بما
93. وجل
94. وإن
95. حاتم
96. مجاهد
97. إبراهيم
98. عمرو
99. وفي
100. منه
101. منهم
102. شيء
103. فلما
104. الدنيا
105. التي
106. إليه
107. ال